<a href="https://colab.research.google.com/github/csrrch/Alura_Google/blob/main/Projeto_Alura_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Projeto: Imersão Alura Google

Objetivo: Este projeto tem como objetivo, criar um resumo e responder questões a respeito de documentos e textos pessoais ou empresariais.

Para este projeto iremos utilizar a técnica da Similaridade do Cosseno

Restrições:
  1. A execução deste projeto se dará apenas para um único arquivo
  2. O arquivo tem que ser .txt

Autor: Cesar Augusto Rocha

Maio / 2024




In [ ]:
# Bibliotecas a serem instaladas
# !pip install nltk
# !pip install pyttsx3

In [38]:
!pip install pyttsx3

In [39]:
import pyttsx3

In [20]:
# Importando bibliotecas

import os
import re
import nltk
import string
import numpy as np
import networkx as nx
from IPython.display import HTML
from nltk.cluster.util import cosine_distance
from sklearn.feature_extraction.text import TfidfVectorizer

# Similaridade do Coseno
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

# Importando variáveis punkt e o pt_core_news_sm
# Para tanto é necessário rodar a primeira vez o download
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# Definicoes das variáveis stopwords e strings
stopwords = nltk.corpus.stopwords.words('portuguese')
strings = string.punctuation

In [22]:
# Biblioteca para ligação com o Google Gemini
import google.generativeai as genai

In [23]:
# Conectando com o Google Gemini
from google.colab import userdata
api_key_colab = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key_colab)



In [24]:
# Atribuindo o caminho do arquivo
path = "/content/drive/MyDrive/Colab Notebooks/Assistente_Alura_Docs/Inteligencia_Artificial_ChatGPT.txt"

In [25]:
# Atribuindo o conteudo contido no documento à variável conteudo
# conteudo = open(path, encoding='utf-8', errors = 'ignore').read().split('\n')
conteudo = open(path, encoding='utf-8', errors = 'ignore').read()


In [26]:
# Definicoes das funcoes

# Preprocessamento do texto, realizando as limpezas necessárias para a tokenização
def preprocessamento(texto):
  texto_formatado = texto.lower()

  # Retirando os espaços em branco
  texto_formatado = re.sub(r'\s+', ' ', texto_formatado)

  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado


In [27]:
#
# Similaridade do Cosseno - Sumarização
#

# Função para calcular a similaridade entre as sentencas
def calcula_similaridade_sentencas(sentenca1, sentenca2):
  palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
  palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]

  todas_palavras = list(set(palavras1 + palavras2))

  vetor1 = [0] * len(todas_palavras)
  vetor2 = [0] * len(todas_palavras)

  for palavra in palavras1:
    vetor1[todas_palavras.index(palavra)] += 1
  for palavra in palavras2:
    vetor2[todas_palavras.index(palavra)] += 1

  return 1 - cosine(vetor1, vetor2)

# Função para calcular a matriz de similaridade enre as sentencas
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
  #print(matriz_similaridade)
  for i in range(len(sentencas)):
    for j in range(len(sentencas)):
      if i == j:
        continue
      matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])

  return matriz_similaridade

# Função para sumarizar o texto
def cosseno_sumarizar(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])

  return sentencas_originais, melhores_sentencas, notas_ordenadas


In [28]:
# Função para responder as perguntas do usuario
def responder(texto_usuario):
    resposta_chatbot = ''
    conteudo_preprocessado = []
    conteudo_preprocessado = sentencas_formatadas
    conteudo_preprocessado.append(texto_usuario)

    tfidf = TfidfVectorizer()
    palavras_vetorizadas = tfidf.fit_transform(conteudo_preprocessado)

    # Pegando a última frase da lista com o [-1]
    similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

    # Receber pegar um valor somente [-2]
    indice_sentenca = similaridade.argsort()[0][-2]
    vetor_similar = similaridade.flatten()
    vetor_similar.sort()

    # retornar o valor encontrado
    vetor_encontrado = vetor_similar[-2]

    if (vetor_encontrado == 0):
        resposta_chatbot = resposta_chatbot + 'Desculpe, mas não entendi!'
        return resposta_chatbot
    else:
        # print(indice_sentenca)
        resposta_chatbot = resposta_chatbot + sentencas_originais[indice_sentenca]
        return resposta_chatbot


In [29]:
# Configurando o modelo Gemini

# candidate_count: qtde de informacoes de retorno
# temperature: 0 - 1 = Aleatoreidade da retorno
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

# configuracoes de seguranca (safety settings)
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [30]:
# Iniciando o modelo
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [31]:
# Criando o Chat

chat = model.start_chat(history=[])

In [32]:
# Código da Aplicação para criação das variáveis a serem utilizadas pelo Assistente

# Fazendo a limpeza do conteudo e atribuindo a variavel texto_processado
texto_processado = preprocessamento(conteudo)

In [33]:
# Gerando duas variaveis:
# sentecas_originais = variavel tipo lista contendo o texto origial do documento
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(conteudo)]
# sentencas_formadadas = variavel tipo lista contendo o texto tokenizado
sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]


In [34]:
# Escolhi 20% das sentenças para aplicar a sumarização
qtd_sentenca = int(len(sentencas_originais)*0.20 + 1)

sentencas_originais, melhores_sentencas, notas_sentencas = cosseno_sumarizar(conteudo, qtd_sentenca)


In [35]:
# Imprimindo o Resumo do conteúdo

display(HTML(f'<h1>Resumo do Arquivo</h1>'))
display(HTML(f'<br>'))
display(HTML(f'<h3>{path}</h3>'))
display(HTML(f'<br>'))

for sentenca in melhores_sentencas:
   display(HTML(f"{sentenca}"))
   display(HTML(f'<br>'))

In [36]:
# Conversação com o assistente

continuar = True
print('Olá, sou um chatbot e vou responder perguntas sobre o arquivo.','\n')
print('Para sair digite: sair','\n')
resposta_chat = ""
resposta_gemini = ""
while (continuar == True):
    print('Faça aqui sua pergunta: ')
    texto_usuario = input()
    texto_usuario = texto_usuario.lower()
    if (texto_usuario != 'sair'):
        resposta_chat = responder(preprocessamento(texto_usuario))
        resposta_gemini = chat.send_message(texto_usuario)

        display(HTML(f'<h1>Resposta Chat</h1>'))
        display(HTML(f'<br>'))
        display(HTML(f'{resposta_chat}'))

        display(HTML(f'<br>'))
        display(HTML(f'<h1>Resposta Gemini</h1>'))
        display(HTML(f'{resposta_gemini.text}'))
        display(HTML(f'<br>'))

    else:
        continuar = False
        print('Chatbot: Até breve!')


Olá, sou um chatbot e vou responder perguntas sobre o arquivo. 

Para sair digite: sair 

Faça aqui sua pergunta: 
o que é pytorch


Faça aqui sua pergunta: 
sair
Chatbot: Até breve!
